### Neural Network Model

In [1]:
# Import Required Libraries

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam


In [2]:
# Load and Inspect the Dataset
data = pd.read_csv('TrainingData.csv')

# Display dataset information
print("Dataset Info:")
print(data.info())

print("\nSample Data:")
print(data.head())


Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   BID        2000 non-null   int64 
 1   Bitstream  2000 non-null   object
 2   class      2000 non-null   int64 
dtypes: int64(2), object(1)
memory usage: 47.0+ KB
None

Sample Data:
   BID                                          Bitstream  class
0    0  1000111010111101101100110111001111001000101111...      1
1    1  1101111100101011111111101101010001110110000010...      1
2    2  0011001010001010100100011101000111110100101111...      0
3    3  1101010110000110100001001100111101000000110001...      1
4    4  1010111100001001000101010010111010011101001100...      1


In [3]:
# Convert Bitstream to Numeric Features

def bitstream_to_features(bitstream):
    return [int(bit) for bit in bitstream]

bit_features = data['Bitstream'].apply(bitstream_to_features).tolist()
X = np.array(bit_features)  # Feature matrix (shape: [n_samples, 1024])
y = data['class']           # Target labels (0 or 1)

print("\nFeature Matrix Shape:", X.shape)
print("First Feature Vector:", X[0])
print("Labels:", y.unique())



Feature Matrix Shape: (2000, 1024)
First Feature Vector: [1 0 0 ... 1 1 0]
Labels: [1 0]


In [4]:
# Train-Test Split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("\nTraining Set Shape:", X_train.shape)
print("Testing Set Shape:", X_test.shape)



Training Set Shape: (1600, 1024)
Testing Set Shape: (400, 1024)


In [5]:
# Define the Neural Network architecture
model = Sequential([
    Dense(512, activation='relu', input_dim=1024),  # First hidden layer
    Dropout(0.3),                                   # Dropout for regularization
    Dense(256, activation='relu'),                  # Second hidden layer
    Dropout(0.3),                                   # Dropout for regularization
    Dense(1, activation='sigmoid')                  # Output layer (binary classification)
])

# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001),  # Optimizer
              loss='binary_crossentropy',           # Loss function for binary classification
              metrics=['accuracy'])                 # Metric to monitor


C:\Python 311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [6]:
# Train the model
history = model.fit(
    X_train, y_train,
    epochs=20,                 # Number of epochs
    batch_size=32,             # Batch size
    validation_split=0.2,      # Use 20% of the training data for validation
    verbose=1                  # Print progress during training
)


Epoch 1/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 4s 26ms/step - accuracy: 0.4970 - loss: 0.8512 - val_accuracy: 0.5219 - val_loss: 0.6938
Epoch 2/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - accuracy: 0.5337 - loss: 0.6877 - val_accuracy: 0.4906 - val_loss: 0.7036
Epoch 3/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 10ms/step - accuracy: 0.5924 - loss: 0.6715 - val_accuracy: 0.4938 - val_loss: 0.7174
Epoch 4/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.6525 - loss: 0.6256 - val_accuracy: 0.4531 - val_loss: 0.7946
Epoch 5/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.7150 - loss: 0.5555 - val_accuracy: 0.5406 - val_loss: 0.7555
Epoch 6/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.7692 - loss: 0.4845 - val_accuracy: 0.5312 - val_loss: 0.9576
Epoch 7/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.7764 - loss: 0.4533 - val_accuracy: 0.4688 - val_loss: 1.0333
Epoch 8/20
40/40 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - accuracy: 0.8603 - loss: 0.3497 - val_accuracy: 0.5219 - val_

In [7]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print("\nTest Set Accuracy:", accuracy)

# Make predictions
y_pred = (model.predict(X_test) > 0.5).astype("int32")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4772 - loss: 2.4624

Test Set Accuracy: 0.5024999976158142
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step

Classification Report:
              precision    recall  f1-score   support

           0       0.50      0.45      0.47       200
           1       0.50      0.56      0.53       200

    accuracy                           0.50       400
   macro avg       0.50      0.50      0.50       400
weighted avg       0.50      0.50      0.50       400

